Step 1: Initialize the Simulation Environment
First, let's set up the basic parameters for our simulation, including the number of cities, scenarios, and the structure for trends and shocks.

In [1]:
import numpy as np

np.random.seed(42)  # For reproducibility

num_cities = 3
num_scenarios = 100

# Initialize arrays to hold the demand for each city across scenarios
demand = np.zeros((num_scenarios, num_cities))


Step 2: Simulate Trends and Shocks
For each city, we'll simulate both a general trend (which could be positive or negative, reflecting economic growth or decline) and random shocks that could represent unexpected events affecting supply and demand.

In [2]:
# Trends - Let's assume a simple linear trend for simplicity, which can vary across cities
trend_strengths = np.random.uniform(-0.02, 0.02, num_cities)  # Some cities might grow, others might decline

# Shocks - Random events affecting each city, with occasional major shocks
shock_strength = 0.1  # Standard deviation of normal shocks
major_shock_strength = 0.5  # Magnitude of major shocks
major_shock_prob = 0.1  # Probability of a major shock in any scenario

for city in range(num_cities):
    # Apply the trend for each city
    demand[:, city] += np.linspace(0, trend_strengths[city]*num_scenarios, num_scenarios)
    
    # Apply normal shocks
    demand[:, city] += np.random.normal(0, shock_strength, num_scenarios)
    
    # Apply major shocks sporadically
    for scenario in range(num_scenarios):
        if np.random.rand() < major_shock_prob:
            demand[scenario, city] += np.random.normal(0, major_shock_strength)

# Apply covariance between cities' demands to simulate economic linkage
cov_matrix = np.array([[1.0, 0.5, 0.3],
                       [0.5, 1.0, 0.4],
                       [0.3, 0.4, 1.0]])  # Simplified example covariance matrix
cholesky_decomp = np.linalg.cholesky(cov_matrix)

demand = demand.dot(cholesky_decomp.T)  # Applying covariance


Step 3: Demand and Supply Adjustment Based on Linkages
Now, let's assume some basic economic linkages between the cities. For instance, excess demand in one city can be partially met by surplus supply in another, depending on the strength of their economic ties.

In [3]:
linkage_matrix = np.array([[1.0, 0.2, 0.1],
                           [0.2, 1.0, 0.3],
                           [0.1, 0.3, 1.0]])  # Example linkage matrix indicating how cities can influence each other

# Adjust demand based on linkages (simplified model)
for scenario in range(num_scenarios):
    for city in range(num_cities):
        # Calculate adjustment based on other cities' surplus/deficit
        adjustment = sum(linkage_matrix[city, other_city] * (demand[scenario, other_city] - demand[scenario, city])
                         for other_city in range(num_cities) if other_city != city)
        demand[scenario, city] += adjustment



To construct a simulation that involves optimizing capital endowment and production decisions in a two-stage stochastic optimization framework for a company operating in three cities, we'll follow these steps closely:

Define the Supply Function for each city, dependent on capital endowment and operational decisions.
Stage 1: Prioritize capital allocation to ensure production capabilities.
Stage 2: Determine the optimal production level based on demand scenarios.
Objective: Ensure demand can be met at least 95% of the time while maximizing profits.
Since the detailed implementation of the algorithm from the provided paper is complex and would require specific knowledge from the paper itself, which we've discussed conceptually but haven't executed directly here, I'll outline a generalized approach based on common principles in two-stage stochastic optimization.

Step 1: Supply Function and Capital Endowment
Let's define a simplified supply function where the production capacity and cost are functions of the capital endowment. We assume higher capital endowment increases production capacity but at a diminishing rate.

In [4]:
def supply(capital, production, city_index):
    """
    Calculate the supply based on capital endowment and production decision.
    city_index allows for city-specific parameters in the supply function.
    """
    capacity = np.log1p(capital[city_index])  # Logarithmic growth of capacity with capital
    cost_per_unit = 1 / (1 + np.log1p(capital[city_index]))  # Decreasing cost per unit with more capital
    supply = min(capacity, production)
    cost = production * cost_per_unit
    return supply, cost

In [5]:
import scipy.optimize


capital_bounds = (0.1, 10)  # Bounds for capital investment decisions
production_bounds = (0, np.inf)  # Production can range from 0 to an upper limit

# Stage 1: Optimizing capital endowment to ensure flexibility in meeting demand
# Simplified as a single optimization across all cities for illustration
def stage1_objective(capital):
    """
    Stage 1 objective: Minimize capital while ensuring potential to meet demand.
    This is a placeholder and should be replaced with a more complex model.
    """
    total_cost = sum(capital)  # Simplify: Minimize total capital endowment
    return total_cost

initial_capital = [1 for _ in range(num_cities)]  # Initial guess for capital endowment
capital_opt = scipy.optimize.minimize(stage1_objective, initial_capital, bounds=[capital_bounds]*num_cities)

# Stage 2: Given capital, optimize production for each scenario to maximize profit
def stage2_objective(production, capital, demand_scenario):
    """
    Stage 2 objective: Maximize profit given demand scenario and capital endowment.
    """
    profit = 0
    for city_index in range(num_cities):
        city_demand = demand_scenario[city_index]
        city_supply, production_cost = supply(capital, production[city_index], city_index)
        revenue = min(city_supply, city_demand) * selling_price_per_unit  # Assume a fixed selling price per unit
        profit += revenue - production_cost
    return -profit  # Negative because we minimize the function

selling_price_per_unit = 2  # Placeholder selling price
production_decisions = []

# Optimize production for a subset of representative scenarios to illustrate
for scenario_index in np.random.choice(range(len(demand)), size=10, replace=False):
    demand_scenario = demand[scenario_index]
    production_opt = scipy.optimize.minimize(stage2_objective, [1 for _ in range(num_cities)], args=(capital_opt.x, demand_scenario), bounds=[production_bounds]*num_cities)
    production_decisions.append(production_opt.x)


Preparing the Dataset for QNN
Assuming demand has been generated per the previous setup and contains scenarios for each city, the first step is preparing this data for training the QNN. Each city's demand series would serve as a target variable, with relevant features extracted or engineered as necessary (e.g., historical demand, economic indicators, etc.). For simplicity, we'll proceed directly to defining and training a QNN model.

Defining the Quantile Neural Network (QNN)
The QNN will be structured to predict quantiles of future demand for each city, capturing the uncertainty inherent in these predictions

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim

class DemandQNN(nn.Module):
    def __init__(self, input_size, output_size):
        super(DemandQNN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, output_size)  # Assuming 3 quantiles per city
        )
    
    def forward(self, x):
        return self.network(x)

# Assuming each city's demand can be predicted based on 5 features
input_size = 3
# Output size: 3 quantiles * num_cities
output_size = 3 * num_cities
model = DemandQNN(input_size=input_size, output_size=output_size)

def quantile_loss(preds, targets, quantiles):
    assert len(quantiles) == preds.shape[1], "Quantiles must match the last dimension of predictions."
    loss = 0
    for i, q in enumerate(quantiles):
        errors = targets - preds[:, i]
        loss += torch.max((q-1) * errors, q * errors).mean()
    return loss / len(quantiles)



Training the QNN
The training process involves using historical data to predict future demand quantiles. A custom loss function, such as the pinball loss, is typically used for quantile regression.

In [7]:
from torch.utils.data import TensorDataset, DataLoader

# Assume quantiles are 25th, 50th, and 75th percentiles
quantiles = [0.25, 0.5, 0.75]
num_epochs = 100
# Correct input_size based on your features (3 for the three cities in this case)
model = DemandQNN(input_size=3, output_size=num_cities * len(quantiles))

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Simplifying assumption: use lagged demand as features
# # This is a simplification; in practice, you might include more sophisticated features
X_demand = demand[:-1]  # Features: demand up to the second-to-last scenario
Y_demand = demand[1:]   # Targets: demand from the second scenario onward

# # Convert to PyTorch tensors for training the QNN
X_train = torch.tensor(X_demand, dtype=torch.float32)
Y_train = torch.tensor(Y_demand, dtype=torch.float32)


    
from torch.utils.data import TensorDataset, DataLoader

# Ensure your DataLoader is set up as before
dataset = TensorDataset(X_train, Y_train)
batch_size = 32  # This can be adjusted based on your system's memory capacity
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    
    for X_batch, Y_batch in dataloader:
        optimizer.zero_grad()
        
        # Forward pass (model and data are already on CPU)
        predictions = model(X_batch)
        
        # Compute loss (ensure your quantile_loss function does not assume GPU computation)
        loss = quantile_loss(predictions, Y_batch, quantiles)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * X_batch.size(0)
    
    # Optional: Print average loss every 10 epochs
    if epoch % 10 == 0:
        avg_loss = total_loss / len(dataloader.dataset)
        print(f'Epoch [{epoch}/{num_epochs}], Average Loss: {avg_loss:.4f}')



: 